In [ ]:
import sys

orig_stdout = sys.stdout
f = open('out.txt', 'w')
sys.stdout = f

sys.stdout = orig_stdout

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
def combine_dict(list_of_dictionaries):
    dic = defaultdict(lambda: list())
    for d in list_of_dictionaries:  
        for item in d.items():
            dic[item[0]].append(item[1])
    return dic
    
def scale(df,col,on_range = True):
    if on_range: # Scaling based on Range
        df[col] -= df[col].min()
        df[col] /= df[col].max()
    else: # Scaling based on SD
        sd = df[col].std()
        df[col] -= df[col].mean()
        df[col] /= sd
    return

In [116]:
fil = pd.read_excel('./Course_Advisory_Data.xlsx')
pre_req = pd.read_excel('./Prerequisites of All Courses.xlsx')
pre_req.set_index('Code',inplace=True)
pre_req.drop(['Sr#','Type','Status','Equivalence'],axis=1,inplace=True)

In [122]:

file = ['Advanced Research Methods',
        'Research in Marketing',
        'MS Thesis - I',
        'MS Thesis - II',
        'PhD Thesis - I',
        'PhD Thesis - II',
        'PhD Thesis - III',
        'PhD Thesis - IV',
        'Applied Programming', 
        'Research Methodology'
        ]

for line in file:
    line = line.replace('\n','')
    indexNames = fil[ fil['Course Title'] == line ].index
    fil.drop(indexNames,inplace=True)

fil.dropna(inplace= True)

In [124]:
cols = fil.columns.to_list()
semesters = list(fil.Semester.unique())
roll_nos = list(fil['Sr. No'].unique())

In [125]:
# Dictionary for Course Code and Course Title and Credit Hours
course_guide = combine_dict([dict(zip(fil['Course Code'], fil['Course Title'])),dict(zip(fil['Course Code'], fil['Credit Hours']))])
# crs = pre_req.index.to_list()
# for c in crs:
#     if c not in course_guide:
#         course_guide[c] = pre_req.ix[c].to_list()[0:2]

# Dictionary for Grade and Grade Point
# grade_dictionary = dict(zip(fil['Grade'], fil['Grade Point']))
grade_dictionary = dict(zip(fil['Grade Point'], fil['Grade']))
fil.dropna(inplace= True)

In [134]:

data = fil
data.set_index('Sr. No',inplace= True)
data = data.drop(labels = ['Grade','Credit Hours'],axis = 1)

roll_nos = data.index.unique().to_list()
# scale(data, 'CGPA')
# scale(data, 'SGPA')
# scale(data, 'Grade Point')

modified_data = pd.DataFrame(columns=['Roll_No'],data=roll_nos)
modified_data.set_index(['Roll_No'],inplace=True)

cols = ['Course Title','Grade_Point','Semester','Warning','SGPA','CGPA']

for c in cols:
    modified_data[c] = np.empty((len(roll_nos), 0)).tolist()

In [136]:
roll_nos.sort()
for roll in roll_nos:
    for semester in semesters:
        semester = str(semester)
        try:
            df = pd.DataFrame(data.ix[roll])
            df = df.groupby(['Semester']).get_group(semester)
            modified_data.ix[roll].Semester.append([semester])
            modified_data.ix[roll]['Course Title'].append(list(df['Course Title'].to_list()))
            modified_data.ix[roll].Grade_Point.append(list(df['Grade Point'].to_list()))
            modified_data.ix[roll].Warning.append(list([df['Warning'].to_list()[0]]))
            modified_data.ix[roll].SGPA.append(list([df['SGPA'].to_list()[0]]))
            modified_data.ix[roll].CGPA.append(list([df['CGPA'].to_list()[0]]))
        except KeyError:
            pass

In [141]:
# modified_data.to_excel('New_Data.xlsx')
modified_data.to_pickle('New_Data_pickle.xlsx')

In [143]:
data = pd.read_pickle('./New_Data_pickle.xlsx')
cols = data.columns.to_list()
students = data.index.to_list()

In [144]:

xyData = pd.DataFrame(columns=['Taken_Courses','Grades','CGPA','SGPA','Warning','Recommended'])

students = data.index.to_list() # Enrolled Students

for student in students:
    student_data = data.loc[student]
    for i in range(len(data.loc[student].Semester)-1):
        # print(student_data['Course Title'])
        xyData = xyData.append({'Taken_Courses':student_data['Course Title'][i],
        'Grades':student_data.Grade_Point[i],
        'CGPA': student_data.CGPA[i],
        'SGPA':student_data.SGPA[i],
        'Warning':student_data.Warning[i],
        'Recommended':student_data['Course Title'][i+1]} ,
        ignore_index=True)
        if len(student_data['Course Title'][i]) != len(student_data.Grade_Point[i]):
            print(len(student_data['Course Title'][i]),len(student_data.Grade_Point[i]))


xyData.to_pickle('xydata.xlsx')

In [145]:
xyData = pd.read_pickle('./xydata.xlsx')
# xyData = xyData.apply(np.random.permutation, axis=1)
# xyData = pd.DataFrame(xyData)

In [146]:
ratio = 0.8
size = int(len(xyData) * ratio)
x_train = xyData.values[:size,:5]
y_train = xyData.values[:size,5:]

# print(x_train.shape,y_train.shape)

x_test = xyData.values[size:,:5]
y_test = xyData.values[size:,5:]

# print(x_test.shape,y_test.shape)

In [167]:

from collections import Counter

class KNN:
	def __init__(self, k=1):
		self.k = k

	def train(self, x, y):
		self.X = x
		self.Y = y

	def predict(self, x_pred):
		scores = []
		for x in self.X:
			score = 999
			# Index(['Taken_Courses', 'Grades', 'CGPA', 'SGPA', 'Warning', 'Recommended'], dtype='object')
			if len(set(x[0])&set(x_pred[0])) != 0:
				score = 0
				common = list(set(x[0])&set(x_pred[0]))
				# print(common)
				for c in common:
					id1 = x_pred[0].index(c)
					id2 = x[0].index(c)
					score += (x_pred[1][id1] - x[1][id2])**2
				score /= len(set(x[0])&set(x_pred[0])) ** 2
				score += ((x_pred[2][0] - x[2][0])**2)
				score += ((x_pred[3][0] - x[3][0])**2)
				score += ((x_pred[4][0] - x[4][0])**2)
			
			scores.append(score)
		scores = np.array(scores)

		print("")
		
		min_vals = np.argmin(scores)
		# print(x_pred[0])
			# l = list(self.Y[:, 0][min_vals])
			# d = dict((x, l.count(x)) for x in set(l))
			# mV = max(d, key=d.get)
			# label.append(mV)
		return (self.X[min_vals],self.Y[min_vals][0])
	def evaluate(self):

		pass

In [168]:
r = 667
classifier = KNN(1)
classifier.train(x_train,y_train)
x,yy = classifier.predict(x_test[r])

print("\n\nGiven\n\n")
for idx,y in enumerate(x_test[r][0]):
    print(y,grade_dictionary.get(x_test[r][1][idx]))

print("\n\nMatched\n\n")
for idx,y in enumerate(x[0]):
    print(y,grade_dictionary.get(x[1][idx]))

print("\n\nLabels\n\n")
for idx,y in enumerate(y_test[r][0]):
    print(y)

print("\n\nPredicted\n\n")
for idx,i in enumerate(yy):
    print(i)





Given


English Composition and Comprehension - Lab B+
Calculus and Analytical Geometry B
Programming Fundamentals A
Intro to Info. and Comm. Technologies A-
Applied Physics C+
Programming Fundamentals - Lab A
English Composition and Comprehension A-
Islamic and Religious Studies C+


Matched


Computer Programming A-
English Composition D
Digital Logic Design - Lab A
Digital Logic Design A
Islamic and Religious Studies C+
Computer Programming Lab B+
Calculus - II A


Labels


Communication & Presentation Skills
Digital Logic Design
Object Oriented Programming - Lab
Differential Equations (Cal II)
Digital Logic Design - Lab
Pakistan Studies
Object Oriented Programming
Communication & Presentation Skills - Lab


Predicted


Pakistan Studies
Fundamentals of Management
Comp. Organization and Assembly Lang.
Data Structures
Discrete Structures
Comp. Organization and Assembly Lang. Lab


In [157]:
grade_dictionary

{4.0: 'A',
 3.0: 'B',
 1.67: 'C-',
 3.33: 'B+',
 3.67: 'A-',
 1.0: 'D',
 2.0: 'C',
 2.67: 'B-',
 2.33: 'C+',
 1.33: 'D+',
 0.0: 'F'}